<a href="https://colab.research.google.com/github/Pugianf/ANTAQ/blob/main/ANTAQ_An%C3%A1lise_IDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# installing librarys

!pip install tabula-py
clear_output()

In [8]:
# importing

import numpy as np
import pandas as pd
import tabula
from IPython.display import clear_output

In [3]:
# connecting drive

from google.colab import drive
drive.mount('/content/drive')
clear_output()

Mounted at /content/drive


In [72]:
# creating an empty DataFrame

df = pd.DataFrame()

In [73]:
# list of IDA years - excepting the first one "2015.2"

IDA_portos_years = [2015, 2016, 2017, 2018, 2019, 2020]

In [74]:
# loop for reading the original data and concat to df

for count, value in enumerate(IDA_portos_years):
  file = f'/content/drive/MyDrive/IDP/Dataset/RelRankingIDA_{count}.pdf'
  table = tabula.read_pdf(file,pages='all')
  frames = [table[0], table[1]]
  final_table = pd.concat(frames)
  final_table['Ano'] = value
  final_table.drop(columns='Instalação', inplace=True)
  final_frame = [df, final_table]
  df = pd.concat(final_frame)

In [78]:
# wrangling the final DataFrame

df.rename(columns={'Unnamed: 0':'Porto'}, inplace=True)

df.set_index('Ano', inplace=True)

df

,Posição,Porto,IDA
Ano,,,
2015,1,São Sebastião,"99,37"
2015,2,Itajaí,"93,74"
2015,3,Itaqui,"89,32"
2015,4,Suape,"83,66"
2015,5,Paranaguá,"83,11"
...,...,...,...
2020,27,Pelotas,"52,70"
2020,28,Porto Velho,"44,68"
2020,29,Forno,"39,63"
